<a href="https://colab.research.google.com/github/gilzeevi25/STTM/blob/main/Extract_data_from_PAN12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Based on a solution found in Pranjalya Tiwari Github account:
https://github.com/Pranjalya/anti-predator-chat-app/tree/f1134b23921964f817a888d3950706f4615a612b/data

```




In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/pan12 dataset

/content/gdrive/MyDrive/pan12 dataset


In [6]:
!unzip -q pan12-sexual-predator-identification-test-corpus-2012-05-21.zip -d extracted_files/
!unzip -q pan12-sexual-predator-identification-training-corpus-2012-05-01.zip -d extracted_files/

In [7]:
!mv extracted_files/pan12-sexual-predator-identification-test-corpus-2012-05-21/ extracted_files/test_corpus
!mv extracted_files/pan12-sexual-predator-identification-training-corpus-2012-05-01/ extracted_files/train_corpus

In [8]:
!cat extracted_files/train_corpus/readme.txt


Overview

This archive contains the training corpus for the "Sexual Predator Identification" task of the PAN 2012 Lab, held in conjunction with the CLEF 2012 conference.

Find out about all the details at http://pan.webis.de.



Training Corpus Description

Update 01 May 2012:

pan12-sexual-predator-identification-training-corpus-2012-05-01.xml A new xml file containing conversations without bad username substitution.
pan12-sexual-predator-identification-diff.txt A text file containing conversation id and line number of modified text 
pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt The list of predators without the ones not present in the traininig set



The corpus comprises:

pan12-sexual-predator-identification-training-corpus.xml An xml file containing around 60000 documents (each document is a conversation)
pan12-sexual-predator-identification-training-corpus-predators.txt A file containing a list of predators id

The xml file is organized as follow:


In [9]:
!ls extracted_files/train_corpus


pan12-sexual-predator-identification-diff.txt
pan12-sexual-predator-identification-training-corpus-2012-05-01.xml
pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt
readme.txt


In [10]:
train_predators = []
with open('extracted_files/train_corpus/pan12-sexual-predator-identification-training-corpus-predators-2012-05-01.txt', 'r') as f:
    train_predators = f.readlines()
    train_predators = [j.rstrip('\n') for j in train_predators]
print("Total number of predators in Training corpus : {}".format(len(train_predators)))

Total number of predators in Training corpus : 142


In [11]:
import os
os.system('pip install xmltodict')
import xmltodict
import json
import pandas as pd

class ExtractText():
    def __init__(self, filename, out_folder='./'):
        '''
        Extract details from XML files
        Args : filename -> Path to the XML file
               out_folder -> Path to output folder
        '''
        self.filename = filename
        if (out_folder[-1]=='/'):
            self.out_folder = out_folder
        else:
            self.out_folder = out_folder + '/'
        
        try:
            os.mkdir(self.out_folder)
        except:
            pass

        print("Parsing XML to Dictionary...")
        dictionary = self.xml_to_dictionary()

        # Converting chat message with single chat to list format
        for i in dictionary['conversations']['conversation']:
            if (str(type(i['message'])) != "<class 'list'>"):
                i['message'] = [i['message']]

        print('Converting XML to JSON format...')
        self.xml_to_json(dictionary)

        print('Converting XML to CSV format...')
        self.xml_to_csv(dictionary)
        print("Files created in {} directory".format(self.out_folder))

    def xml_to_dictionary(self):
        '''
        Converts XML file to data dictionary
        '''
        with open(self.filename) as xml_file:
            data_dict = xmltodict.parse(xml_file.read())
        return data_dict

    def xml_to_json(self, dictionary):
        '''
        Converts parsed dictionary to json and saves
        '''
        data = json.dumps(dictionary)
        with open(self.out_folder + self.filename.split('/')[-1].rstrip('xml') + 'json', 'w') as f:
            f.write(data)

    def xml_to_csv(self, dictionary):
        '''
        Converts parsed dictionary to dataframe and saves in CSV format
        '''
        data = []
        for conv in dictionary['conversations']['conversation']:
            id = conv['@id']
            for message in conv['message']:
                d = dict()
                d = {key: message[key] for key in message.keys()}
                d['@id'] = id
                data.append(d)
        df = pd.DataFrame(data)
        df = df [['@id', '@line', 'author', 'time', 'text']]
        df.to_csv(self.out_folder + self.filename.split('/')[-1].rstrip('xml') + 'csv', index=False)

In [13]:
ExtractText('extracted_files/train_corpus/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml', 'data')
ExtractText('extracted_files/test_corpus/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml', 'data')

Parsing XML to Dictionary...
Converting XML to JSON format...
Converting XML to CSV format...
Files created in data/ directory
Parsing XML to Dictionary...
Converting XML to JSON format...
Converting XML to CSV format...
Files created in data/ directory


In [15]:
%pwd

'/content/gdrive/MyDrive/pan12 dataset'

In [16]:
import pandas as pd
train = pd.read_csv('data/pan12-sexual-predator-identification-training-corpus-2012-05-01.csv')
train.head()

,@id,@line,author,time,text
0,e621da5de598c9321a1d505ea95e6a2d,1,97964e7a9e8eb9cf78f2e4d7b2ff34c7,03:20,Hola.
1,e621da5de598c9321a1d505ea95e6a2d,2,0158d0d6781fc4d493f243d4caa49747,03:20,hi.
2,e621da5de598c9321a1d505ea95e6a2d,3,0158d0d6781fc4d493f243d4caa49747,03:20,whats up?
3,e621da5de598c9321a1d505ea95e6a2d,4,97964e7a9e8eb9cf78f2e4d7b2ff34c7,03:20,not a ton.
4,e621da5de598c9321a1d505ea95e6a2d,5,97964e7a9e8eb9cf78f2e4d7b2ff34c7,03:20,you?


In [19]:
def check_predator_author(author_id):
    '''
    Function to return new column for whether the author is a predator or not
    '''
    if author_id in train_predators:
        return 1
    return 0

train['sexual_predator'] = train['author'].apply(check_predator_author)


In [23]:
train['sexual_predator'].value_counts()

0    862629
1     40978
Name: sexual_predator, dtype: int64

In [22]:
print("Total conversations : {}".format(train['@id'].nunique()))


Total conversations : 66927


In [24]:
from tqdm import tqdm

group = train.groupby('@id')
result = pd.DataFrame(columns=['@id', '@line', 'author', 'time', 'text', 'sexual_predator', 'predator_conversation'])

for _, df in tqdm(group):
    if (df['sexual_predator'].any()):
        df['predator_conversation'] = 1
    else:
        df['predator_conversation'] = 0
    result = pd.concat([result, df])

result.info()

100%|██████████| 66927/66927 [52:33<00:00, 21.22it/s]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 903607 entries, 13370 to 280938
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   @id                    903607 non-null  object
 1   @line                  903607 non-null  object
 2   author                 903607 non-null  object
 3   time                   903607 non-null  object
 4   text                   900628 non-null  object
 5   sexual_predator        903607 non-null  object
 6   predator_conversation  903607 non-null  object
dtypes: object(7)
memory usage: 55.2+ MB


In [46]:
result.to_csv('train_inclusive.csv', index=False)

# Deal with test

In [50]:
test_predators_p1 = []
with open('extracted_files/test_corpus/pan12-sexual-predator-identification-groundtruth-problem1.txt', 'r') as f:
    test_predators_p1 = f.readlines()
    test_predators_p1 = [j.rstrip('\n') for j in test_predators_p1]
print("Total number of predators in Test corpus : {}".format(len(test_predators_p1)))
test_predators_p2 = []
with open('extracted_files/test_corpus/pan12-sexual-predator-identification-groundtruth-problem2.txt', 'r') as f:
    test_predators_p2 = f.readlines()
    test_predators_p2 = [j.rstrip('\n') for j in test_predators_p2]
print("Total number of Suspicious in Test corpus : {}".format(len(test_predators_p2)))

Total number of predators in Test corpus : 254
Total number of Suspicious in Test corpus : 6478


In [51]:
test = pd.read_csv('data/pan12-sexual-predator-identification-test-corpus-2012-05-17.csv')
test.head()

,@id,@line,author,time,text
0,affc2df0951b733d14ba92d19d9b7695,1,0a39f78bcb297ab0ebe8a29c28bfed89,15:24,bugmail: [Bug 6978] New: Mark eof-terminated s...
1,affc2df0951b733d14ba92d19d9b7695,2,60659cfda992013e610f285c46692d28,15:32,"Henri, can I ask you a Firefox build question ..."
2,affc2df0951b733d14ba92d19d9b7695,3,b8810fee2f4a71f849f3f7409546d1d9,15:34,"60659cfda992013e610f285c46692d28: sure, but I ..."
3,affc2df0951b733d14ba92d19d9b7695,4,60659cfda992013e610f285c46692d28,15:35,"It appears the build runs through, it creates ..."
4,affc2df0951b733d14ba92d19d9b7695,5,60659cfda992013e610f285c46692d28,15:35,"when I start it, I get my standard install of ..."


In [64]:
def check_predator_author_test(author_id):
    '''
    Function to return new column for whether the author is a predator or not
    '''
    if author_id in test_predators_p1:
        return 1
    return 0
def check_suspicious_conv_test(conv_id_line_id):
    '''
    Function to return new column for whether conversations id and lines id of those line considered suspicious (of a perverted behavior)
    '''
    if conv_id_line_id in test_predators_p2:
        return 1
    return 0

test['sexual_predator'] = test['author'].apply(check_predator_author_test)
test['suspicious'] = test.apply(lambda x: check_suspicious_conv_test(x['@id']+'\t'+str(x['@line'])),axis = 1)

## Test transformed data

In [78]:
assert test[test['sexual_predator'] == 1].author.unique().shape[0] == 254
assert len(test[test['suspicious']== 1]) == 6478

In [79]:
assert result[result['sexual_predator'] == 1].author.unique().shape[0] == 142

In [82]:
test.to_csv('test_inclusive.csv', index=False)

In [83]:
pd.read_csv('test_inclusive.csv')

,@id,@line,author,time,text,sexual_predator,suspicious
0,affc2df0951b733d14ba92d19d9b7695,1,0a39f78bcb297ab0ebe8a29c28bfed89,15:24,bugmail: [Bug 6978] New: Mark eof-terminated s...,0,0
1,affc2df0951b733d14ba92d19d9b7695,2,60659cfda992013e610f285c46692d28,15:32,"Henri, can I ask you a Firefox build question ...",0,0
2,affc2df0951b733d14ba92d19d9b7695,3,b8810fee2f4a71f849f3f7409546d1d9,15:34,"60659cfda992013e610f285c46692d28: sure, but I ...",0,0
3,affc2df0951b733d14ba92d19d9b7695,4,60659cfda992013e610f285c46692d28,15:35,"It appears the build runs through, it creates ...",0,0
4,affc2df0951b733d14ba92d19d9b7695,5,60659cfda992013e610f285c46692d28,15:35,"when I start it, I get my standard install of ...",0,0
...,...,...,...,...,...,...,...
2058776,8deed7c66340728e6863f3d931d4cca7,53,105ca6d0fd6c2c3e136980d3548f16ab,02:32,"Come the revolution, I'll worry about it",0,0
2058777,8deed7c66340728e6863f3d931d4cca7,54,48a4fba2c4916bbc8aae694a3877dca3,02:33,no pasaran!,0,0
2058778,8deed7c66340728e6863f3d931d4cca7,55,48a4fba2c4916bbc8aae694a3877dca3,02:33,:),0,0
2058779,8deed7c66340728e6863f3d931d4cca7,56,e1e3d026bb7ee9ee264316da10378aeb,02:35,25mhz is slow :(,0,0
